# Steph's Part

### Loading Required libraries

In [1]:
##Import Key Libraries for use
import datetime as dt
import pandas as pd
import numpy as np
import sklearn
#My add
from sklearn.linear_model import Lasso
from sklearn.model_selection import train_test_split
from sklearn.metrics import explained_variance_score, mean_absolute_error, r2_score, mean_squared_error
#from math import sqrt

### Reading the Data and Performing Basic Data Checks

In [2]:
#Import CSV
covid19Dataframe = pd.read_csv("https://raw.githubusercontent.com/WhipSnake23/Python-Class-Project/main/Data/owid-covid-data.csv")
#Filter Dataset to Canada
covid19Dataframe = covid19Dataframe[covid19Dataframe.iso_code.str.contains("CAN")]
#Remove Smoothed Columns from dataframe
covid19Dataframe = covid19Dataframe.drop(columns=['new_cases_smoothed','new_deaths_smoothed','new_cases_smoothed_per_million','new_deaths_smoothed_per_million','new_tests_smoothed','new_tests_smoothed_per_thousand','new_vaccinations_smoothed','new_vaccinations_smoothed_per_million'])
#Reset Index
covid19Dataframe.reset_index(drop=True, inplace=True)
#covid19Dataframe.dropna(inplace=True)
covid19Dataframe.dropna(subset=['total_deaths'],inplace=True)

In [3]:
# Is there value in this view?
print(covid19Dataframe.shape)
covid19Dataframe.describe()

(350, 51)


,total_cases,new_cases,total_deaths,new_deaths,total_cases_per_million,new_cases_per_million,total_deaths_per_million,new_deaths_per_million,reproduction_rate,icu_patients,...,gdp_per_capita,extreme_poverty,cardiovasc_death_rate,diabetes_prevalence,female_smokers,male_smokers,handwashing_facilities,hospital_beds_per_thousand,life_expectancy,human_development_index
count,350.000000,350.000000,350.000000,350.000000,350.000000,350.000000,350.000000,350.000000,346.000000,347.000000,...,3.500000e+02,350.0,3.500000e+02,3.500000e+02,350.0,3.500000e+02,0.0,350.0,3.500000e+02,3.500000e+02
mean,249064.948571,2430.494286,9757.748571,61.928571,6599.118020,64.397334,258.537111,1.640771,1.133757,336.896254,...,4.401759e+04,0.5,1.055990e+02,7.370000e+00,12.0,1.660000e+01,NaN,2.5,8.243000e+01,9.290000e-01
std,246204.994448,2487.647083,5364.976189,58.621539,6523.341916,65.911627,142.148132,1.553285,0.362303,270.765572,...,1.457275e-11,0.0,4.411672e-13,6.404040e-14,0.0,6.048260e-14,NaN,0.0,2.276992e-13,4.336069e-15
min,77.000000,2.000000,1.000000,0.000000,2.040000,0.053000,0.026000,0.000000,0.760000,0.000000,...,4.401759e+04,0.5,1.055990e+02,7.370000e+00,12.0,1.660000e+01,NaN,2.5,8.243000e+01,9.290000e-01
25%,95438.500000,480.750000,8136.000000,8.000000,2528.698000,12.737500,215.568000,0.212000,0.910000,78.500000,...,4.401759e+04,0.5,1.055990e+02,7.370000e+00,12.0,1.660000e+01,NaN,2.5,8.243000e+01,9.290000e-01
50%,130403.000000,1332.000000,9189.000000,44.000000,3455.102000,35.292000,243.468000,1.166000,1.070000,249.000000,...,4.401759e+04,0.5,1.055990e+02,7.370000e+00,12.0,1.660000e+01,NaN,2.5,8.243000e+01,9.290000e-01
75%,355230.250000,3983.500000,11881.500000,104.000000,9412.028000,105.544750,314.807000,2.756000,1.180000,540.000000,...,4.401759e+04,0.5,1.055990e+02,7.370000e+00,12.0,1.660000e+01,NaN,2.5,8.243000e+01,9.290000e-01
max,850737.000000,16141.000000,21675.000000,244.000000,22540.763000,427.665000,574.292000,6.465000,2.630000,900.000000,...,4.401759e+04,0.5,1.055990e+02,7.370000e+00,12.0,1.660000e+01,NaN,2.5,8.243000e+01,9.290000e-01


### Step 3 - Creating Arrays for the Features and the Response Variable.

In [4]:
target = ["total_deaths"]
features = ["total_cases", "new_cases", "median_age", "gdp_per_capita"]

# https://www.pluralsight.com/guides/linear-lasso-ridge-regression-scikit-learn
#step from website. Do we need to do this?
covid19Dataframe[features] = covid19Dataframe[features]/covid19Dataframe[features].max()
#covid19Dataframe.describe

### Step 4 - Creating the Training and Test Datasets

In [5]:
X = covid19Dataframe[features].values
y = covid19Dataframe[target].values

#Do we need to make the same across all regression types?
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=40)
print(X_train.shape); print(X_test.shape)

(245, 4)
(105, 4)


### Step 5 - Build, Predict and Evaluate the Regression Model

In [6]:
covid_lasso = Lasso()
covid_lasso.fit(X_train, y_train) 
pred_train_lasso= covid_lasso.predict(X_train)

#print(np.sqrt(mean_squared_error(y_train,pred_train_lasso)))
#print(r2_score(y_train, pred_train_lasso))

pred_test_lasso= covid_lasso.predict(X_test)
print(f"Score: {explained_variance_score(y_test, pred_test_lasso):.2f}")
print(f"MAE: {mean_absolute_error(y_test, pred_test_lasso):.2f}")
print(f"RMSE: {np.sqrt(mean_squared_error(y_test, pred_test_lasso)):.2f}")
print(f"r2: {r2_score(y_test, pred_test_lasso):.2f}")

Score: 0.84
MAE: 1709.91
RMSE: 2286.61
r2: 0.83
